#TC Data Download and Convertion

This script downloads the SMOS / SMAP / SAR data from the Cyclobs API and converts the resulting .nc files into .jpg files.

##Imports and configurations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# getting in the directory 
%ls
# insert your path
%cd /content/drive/My\ Drive/ESRIN_PhiLab/Tropical_Cyclones/data
%ls

In [ ]:
#!pip install imgaug==0.2.6
#!pip uninstall shapely
#!pip install shapely --no-binary shapely
#!apt-get install libproj-dev proj-data proj-bin  
#!apt-get install libgeos-dev  
#!pip install cython  
#!pip install cartopy  
#!pip install geoviews
#!pip install rasterio
!pip install netcdf4
#!pip install rioxarray

In [ ]:
# general import
#import bokeh.io
#bokeh.io.output_notebook()
#import geoviews as gv
#import geoviews.feature as gf
#gv.extension('bokeh','matplotlib')
import pandas as pd
import numpy as np
import xarray as xr
#import rasterio as rio
#import rioxarray # geospatial extension for xarray
import os
import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import cartopy
from tqdm.auto import tqdm
import netCDF4
import glob
import argparse
import cv2
from google.colab.patches import cv2_imshow
import math 
from math import radians, cos, sin, asin, sqrt
from dateutil import parser
import datetime
import sys
import cv2
import re
import random

[DEPRECATED] General settings to be changes accordingly.

In [ ]:
"""
# insert here category, mission and feature you want to extract
CATEGORY = "cat1" # either 1, 2, 3, 4, 5
MISSION = "smap" # either sar, smap or smos
if MISSION == "sar":
    KEY = "s1"
    FEATURE_TO_SAVE = "nrcs_detrend"
    CHANNELS = ["nrcs_detrend_cross", "nrcs_detrend_co"]
elif MISSION == "smap":
    KEY = "*smap"
    FEATURE_TO_SAVE = "wind_speed"
else: # smos
    KEY = "SM"
    FEATURE_TO_SAVE = "wind_speed"

# set download path
download_path = CATEGORY   # Tropical_Cyclones/data/cat1
os.makedirs(download_path, exist_ok=True)

# set saving directory accordingly
SAVING_DIRECTORY = "{}/{}_{}_{}".format(CATEGORY, FEATURE_TO_SAVE, MISSION, CATEGORY)   # cat1/nrcs_detrend_sar_cat1
os.makedirs(SAVING_DIRECTORY, exist_ok=True)

# set format of output images
SAVE_FORMAT = "png"
"""

##1.Download the .nc files from the Cyclobs API

In [ ]:
# doanload path
download_path = "SAR_swath_nc/category56"
os.makedirs(download_path, exist_ok=True)

# make the resquest using cyclobs API and store the result in a pandas dataframe
#request_url="https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-2&mission=SMAP,SMOS,S1B,S1A&include_cols=all"
#request_url="https://cyclobs.ifremer.fr/app/api/getData?sid=al122017&instrument=C-Band_SAR&product_type=gridded&include_cols=all"
#request_url="https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-3&mission=SMAP,SMOS,S1B,S1A&include_cols=all"
#request_url="https://cyclobs.ifremer.fr/app/api/getData?mission=S1B,S1A&product_type=swath&include_cols=all"
#request_url="https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-4&cat_max=cat-5&mission=S1B,S1A&product_type=swath&include_cols=all"
request_url="https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-5&mission=S1B,S1A&product_type=swath&include_cols=all"
df_request = pd.read_csv(request_url)

# add download path
df_request['path'] = df_request['data_url'].map(lambda x : os.path.join(download_path,os.path.basename(x)))
#df_request

# download 'data_url' to 'path' with wget, and read files
projection=ccrs.Mercator()
datasets = []
for idx,entry in tqdm(df_request.iterrows(),total=df_request.shape[0]):
    ret = os.system('cd %s ; wget -N  %s' % (os.path.dirname(entry['path']),entry['data_url']))

    if ret == 0 : 
        ds = xr.open_dataset(entry['path'])
        datasets.append(ds)
        #datasets.append(ds.rio.reproject(projection.proj4_params))
        
    else:
        datasets.append(None) # error fetching file

#print(datasets)
df_request['dataset'] = datasets

"""
df_request['dataset'].iloc[0]['wind_speed']

gv_list=[gf.coastline.opts(projection=projection)]
for ds in df_request['dataset']:
    print(ds)
    gv_list.append(gv.Image(ds['wind_speed'].squeeze()[::5,::5],crs=projection).opts(cmap='jet',tools=['hover']))
    
gv.Overlay(gv_list).options(width=800, height=500)
"""

###1.1. Download data_url, sid and name of each Tropical Cyclone

In [ ]:
download_path = "TC_5_try_additional_info"
os.makedirs(download_path, exist_ok=True)

request_url = "https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-5&mission=SMAP,SMOS,S1B,S1A&include_cols=all"
df_request = pd.read_csv(request_url)

# add download path
df_request['path'] = df_request['data_url'].map(lambda x : os.path.join(download_path,os.path.basename(x)))
TC_CyclObs_information = pandas.DataFrame(data = {'data_url': df_request['data_url'], 'sid': df_request["sid"], 'TC_name': df_request["cyclone_name"]})
TC_CyclObs_information["data_url"] = TC_CyclObs_information["data_url"].apply(lambda x: os.path.basename(x))
print(TC_CyclObs_information)
TC_CyclObs_information.to_csv('{}/Cyclobs_info_names.csv'.format(download_path), index=False, header=True)  

# download 'data_url' to 'path' with wget, and read files
#projection=ccrs.Mercator()
#datasets = []
#for idx,entry in tqdm(df_request.iterrows(),total=df_request.shape[0]):
#    ret = os.system('cd %s ; wget -N  %s' % (os.path.dirname(entry['path']),entry['data_url']))
#    if ret == 0 : 
#        ds = xr.open_dataset(entry['path'])
#        datasets.append( ds)
        #datasets.append( ds.rio.reproject(projection.proj4_params))
#    else:
#        datasets.append(None) # error fetching file

###1.2. [DEPRECATED] Convert the information we need to SAVE_FORMAT to be able to use computer vison algorithms

In [ ]:
# convert information relative to SAR mission
if MISSION != "sar":
    sys.exit('The script is only for SAR!')

nc_list = glob.glob("{}/{}*.nc".format(CATEGORY, KEY))

for nc_image in nc_list:
    for FEATURE_TO_SAVE in CHANNELS:
        # read nc image
        full_info_image = netCDF4.Dataset(nc_image, mode='r') 
    
        # getting the information of the feature you want
        try:
            feature_extracted = full_info_image.variables[FEATURE_TO_SAVE][:]
            tmax_units  = full_info_image.variables['time'].units
            feature_extracted = feature_extracted[0]
            full_info_image.close()
        except:
            print(FEATURE_TO_SAVE + " data not available")
            full_info_image.close()
            continue
      
    # save image in SAVE_FORMAT
    if FEATURE_TO_SAVE == "nrcs_detrend_cross":
        plt.imsave("{}/{}_cross.{}".format(SAVING_DIRECTORY, tmax_units[11:], SAVE_FORMAT), 
                   feature_extracted, format = SAVE_FORMAT)
    else:
        plt.imsave("{}/{}.{}".format(SAVING_DIRECTORY, tmax_units[11:], SAVE_FORMAT), 
                   feature_extracted, format = SAVE_FORMAT)

In [ ]:
# convert information relative to SMOS and SMAP missions, substitute KEY with *smap*.nc or *OPER_MIR*.nc
nc_list = glob.glob("{}/{}*.nc".format(CATEGORY, KEY))

for nc_image in nc_list:
    # read nc image
    full_info_image = netCDF4.Dataset(nc_image, mode='r') 

    # getting the information of the feature you want
    feature_extracted = full_info_image.variables[FEATURE_TO_SAVE][:]
    tmax_units  = full_info_image.measurement_start_date
    full_info_image.close()

    # save image in SAVE_FORMAT
    plt.imsave("{}/{}.{}".format(SAVING_DIRECTORY, tmax_units[11:], SAVE_FORMAT), 
               feature_extracted, format = SAVE_FORMAT)

##2.Save the features we need into an image (3 channels = 3 features)

In [ ]:
with_land_mask = True
categories = ["category1", "category2", "category3", "category4", "category5"]


for category in categories:
    #category = 'category1'
    # directory where .nc files are
    saved_dir = "SAR_swath_nc/{}".format(category)

    # directory where corresponding images will be saved
    images_dir = "SAR_swath_images_VV+VH+WS_land=1/{}".format(category)
    #images_dir = "SAR_swath_Vmax/{}".format(category)
    #images_dir = "SAR_swath_masks/{}".format(category)
    os.makedirs(images_dir, exist_ok=True)

    #lon_dir = images_dir + "/lon"
    #os.makedirs(lon_dir, exist_ok=True)
    #lat_dir = images_dir + "/lat"
    #os.makedirs(lat_dir, exist_ok=True)

    nc_list = glob.glob("{}/*.nc".format(saved_dir))
    count=0

    for nc_image in nc_list:
        # read nc image
        full_info_image = netCDF4.Dataset(nc_image, mode='r') 

        try:
            count += 1
            # getting the information of the feature you want
            if with_land_mask:
                # extract mask flag
                mask = full_info_image.variables["mask_flag"][:]
                #mask[mask != 0] = 1

                # dilate mask
                #kernel = np.ones((11, 11), np.int8)
                #mask_dilated = cv2.dilate(mask[0], kernel, iterations = 1)
                #print(np.unique(mask[0]))
                #plt.imshow(mask[0])
                #plt.show()

            feature_co = full_info_image.variables["nrcs_detrend_co"][:]
            if with_land_mask:
                # mask out the land values
                feature_co[0][mask[0] != 0] = 0
                feature_co[0][mask[0] == 1] = 1
                #feature_co[0][mask_dilated != 0] = 0
            feature_co = (feature_co[0] - np.min(feature_co[0]))/(np.max(feature_co[0]) - np.min(feature_co[0]))
            #print("feature_co normalised:", np.max(feature_co), np.min(feature_co))
            if count <5:
                print("feature_co")
                plt.imshow(feature_co)
                plt.show()

            feature_cross = full_info_image.variables["nrcs_detrend_cross"][:]
            if with_land_mask:
                # mask out the land values
                feature_cross[0][mask[0] != 0] = 0
                feature_cross[0][mask[0] == 1] = 1
                #feature_cross[0][mask_dilated != 0] = 0 
            feature_cross = (feature_cross[0] - np.min(feature_cross[0]))/(np.max(feature_cross[0]) - np.min(feature_cross[0]))
            #print("feature_cross normalised:", np.max(feature_cross), np.min(feature_cross))
            if count <5:
                print("feature_cross")
                plt.imshow(feature_cross)
                plt.show()

            feature_wind = full_info_image.variables["wind_speed"][:]
            if with_land_mask:
                # mask out the land values
                feature_wind[0][mask[0] != 0] = 0
                feature_wind[0][mask[0] == 1] = 1
                #feature_wind[0][mask_dilated != 0] = 0 
            feature_wind = (feature_wind[0] - np.min(feature_wind[0]))/(np.max(feature_wind[0]) - np.min(feature_wind[0]))
            #print("feature_wind normalised:", np.max(feature_wind), np.min(feature_wind))
            if count <5:
                print("feature_wind")
                plt.imshow(feature_wind)
                plt.show()

            feature_wso = full_info_image.variables["wind_streaks_orientation"][:]
            #print(np.nanmax(feature_wso[0]), np.nanmin(feature_wso[0]))
            #if np.isnan(feature_wso).any():
            #    count +=1
            #print("feature_wso")
            #plt.imshow(feature_wso[0])
            #plt.show()
            #feature_wso = np.nan_to_num(feature_wso[0])
            #sin_feature_wso = np.sin(feature_wso * np.pi/180.)   # values from -1 to 1
            #sin_feature_wso = (sin_feature_wso - np.min(sin_feature_wso))/(np.max(sin_feature_wso) - np.min(sin_feature_wso))
            #cos_feature_wso = np.cos(feature_wso * np.pi/180.)   # values from -1 to 1
            #cos_feature_wso = (cos_feature_wso - np.min(cos_feature_wso))/(np.max(cos_feature_wso) - np.min(cos_feature_wso))

            # extract longitude and latitude features
            feature_lon = full_info_image.variables["lon"][:]
            feature_lat = full_info_image.variables["lat"][:]

            # extract time registered
            tmax_units = datetime.datetime.strptime(full_info_image.measurementDate, '%Y-%m-%dT%H:%M:%SZ')
            tmax_units = tmax_units.strftime('%Y-%m-%d %H_%M_%S')

            # stack matrices along 3rd axis (depth-wise)
            output_image = np.dstack((feature_co, feature_cross, feature_wind))
            #output_image = np.dstack((feature_wind[0], np.sin(feature_wso[0] * np.pi/180.), np.cos(feature_wso[0] * np.pi/180.)))

            if (output_image < 0).any():
                output_image = np.clip(output_image, a_min=0, a_max=None)
            #print(output_image.shape)
            #print("output_image:", np.max(output_image), np.min(output_image))
            #print("output:", np.nanmax(output_image), np.nanmin(output_image))
            #print("first output image")
            if count <5:
                print("output image")
                plt.imshow(output_image)
                plt.show()

            full_info_image.close()
        except KeyError as err:
            # creating KeyError instance for book keeping
            print("Error:", err)
            full_info_image.close()
            continue
        
        # [DEPRECATED] mask out the land values
        #output_image[mask[0] != 0] = 0
        #print("output image masked")
        #print("after masking:", np.max(output_image), np.min(output_image))
        #plt.imshow(output_image)
        #plt.show()

        # [DEPRECATED] normalise values between [0 ... 1]
        #norm_image = output_image/np.nanmax(output_image)
        #print("after normalising:", np.max(norm_image), np.min(norm_image))
        #print("output image normalised")
        #plt.imshow(norm_image)
        #plt.show()

        # save information extracted
        #cv2.imwrite("{}/{}.png".format(images_dir, tmax_units), output_image)
        plt.imsave("{}/{}.png".format(images_dir, tmax_units), output_image, format = 'png')
        #np.save("{}/{}_mask.npy".format(images_dir, tmax_units), np.array(mask[0]))
        #np.save("{}/{}_Vmax.npy".format(images_dir, tmax_units), np.max(feature_wind[0]))
        #np.save("{}/{}.npy".format(lon_dir, tmax_units), np.mean(feature_lon[0]))
        #np.save("{}/{}.npy".format(lat_dir, tmax_units), np.mean(feature_lat[0]))
    #print(count)

###2.1. Test features

In [ ]:
# directory where nc files are
saved_dir = "SAR_swath_nc/category1"

# directory where corresponding images will be saved
#images_dir = "SAR_swath_images/category1"
#images_dir = "SAR_swath_masks/category5"
#os.makedirs(images_dir, exist_ok=True)
#lon_dir = images_dir + "/lon"
#os.makedirs(lon_dir, exist_ok=True)
#lat_dir = images_dir + "/lat"
#os.makedirs(lat_dir, exist_ok=True)
test_dir = 'test'
os.makedirs(test_dir, exist_ok=True)

nc_list = glob.glob("{}/*.nc".format(saved_dir))
count=0

for nc_image in nc_list[:5]:
    # read nc image
    full_info_image = netCDF4.Dataset(nc_image, mode='r') 

    # getting the information of the feature you want
    try:
        # extract mask flag
        mask = full_info_image.variables["mask_flag"][:]

        feature_co = full_info_image.variables["nrcs_detrend_co"][:]
        #print("feature_co: ", np.max(feature_co[0]), np.min(feature_co[0]))
        #plt.imshow(feature_co[0])
        #plt.show()
        test = feature_co[0]
        test[mask[0] != 0] = 0
        feature_co[0][mask[0] != 0] = 0
        print("feature_co masked:", np.max(feature_co[0]), np.min(feature_co[0]))
        print("test masked:", np.max(test), np.min(test))

        feature_co = (feature_co[0] - np.min(feature_co[0]))/(np.max(feature_co[0]) - np.min(feature_co[0]))
        print("feature_co normalised:", np.max(feature_co), np.min(feature_co))
        #feature_co[mask[0] != 0] = 0
        #print("feature_co masked:", np.max(feature_co), np.min(feature_co))
        plt.imshow(feature_co)
        plt.show()
    
  
        feature_cross = full_info_image.variables["nrcs_detrend_cross"][:]
        #print("feature_cross:", np.max(feature_cross[0]), np.min(feature_cross[0]))
        #plt.imshow(feature_cross[0])
        #plt.show()

        feature_cross = (feature_cross[0] - np.min(feature_cross[0]))/(np.max(feature_cross[0]) - np.min(feature_cross[0]))
        print("feature_cross normalised:", np.max(feature_cross), np.min(feature_cross))
        feature_cross[mask[0] != 0] = 0
        print("feature_cross masked:", np.max(feature_cross), np.min(feature_cross))
        plt.imshow(feature_cross)
        plt.show()


        feature_wind = full_info_image.variables["wind_speed"][:]
        #print("feature_wind:", np.max(feature_wind[0]), np.min(feature_wind[0]))
        #plt.imshow(feature_wind[0])
        #plt.show()


        feature_wind = (feature_wind[0] - np.min(feature_wind[0]))/(np.max(feature_wind[0]) - np.min(feature_wind[0]))
        print("feature_wind normalised:", np.max(feature_wind), np.min(feature_wind))
        feature_wind[mask[0] != 0] = 0
        print("feature_wind masked:", np.max(feature_wind), np.min(feature_wind))
        plt.imshow(feature_wind)
        plt.show()


        feature_wso = full_info_image.variables["wind_streaks_orientation"][:]
        #if np.isnan(feature_wso).any():
        #  count +=1
        #print(np.nanmax(feature_wso[0]), np.nanmin(feature_wso[0]))
        #print("feature_wso")
        #plt.imshow(feature_wso[0])
        #plt.show()

        feature_lon = full_info_image.variables["lon"][:]
        feature_lat = full_info_image.variables["lat"][:]

        tmax_units = datetime.datetime.strptime(full_info_image.measurementDate, '%Y-%m-%dT%H:%M:%SZ')
        tmax_units = tmax_units.strftime('%Y-%m-%d %H_%M_%S')

        # stack matrices along 3rd axis (depth-wise)
        output_image = np.dstack((feature_co, feature_cross, feature_wind))
        if (output_image < 0).any():
            output_image = np.clip(output_image, a_min=0, a_max=None)
        #print(output_image.shape)
        print("output:", np.max(output_image), np.min(output_image))
        #print("first output image")
        #plt.imshow(output_image)
        #plt.show()

        full_info_image.close()

    except KeyError as err:
        # creating KeyError instance for book keeping
        print("Error:", err)
        full_info_image.close()
        continue
    
    # mask out the land values
    #output_image[mask[0] != 0] = 0
    #print("output image masked")
    #print("after masking:", np.max(output_image), np.min(output_image))
    #plt.imshow(output_image)
    #plt.show()

    # normalise values between [0 ... 1]
    #norm_image = output_image/np.nanmax(output_image)
    #print("after normalising:", np.max(norm_image), np.min(norm_image))
    #print("output image normalised")
    #plt.imshow(norm_image)
    #plt.show()

    # save information extracted
    #cv2.imwrite("{}/{}.png".format(images_dir, tmax_units), output_image)
    plt.imsave("{}/{}.png".format(test_dir, tmax_units), output_image, format = 'png')
    #plt.imsave("{}/{}_mask.png".format(images_dir, tmax_units), mask, format = 'png')
    #np.save("{}/{}_mask.npy".format(images_dir, tmax_units), np.array(mask[0]))
    #np.save("{}/{}.npy".format(lon_dir, tmax_units), np.mean(feature_lon[0]))
    #np.save("{}/{}.npy".format(lat_dir, tmax_units), np.mean(feature_lat[0]))

    print("--------------------- Test -----------------------")
    a = cv2.imread("{}/{}.png".format(test_dir, tmax_units))
    #print("cv2_imshow")
    #cv2_imshow(a)
    print("plt.imshow: all")
    plt.imshow(a)
    plt.show()

    #print("feature_co")
    #print("cv2_imshow")
    #cv2_imshow(a[:,:,0])

    print("plt.imshow")
    plt.imshow(a[:,:,0])
    plt.show()
    plt.imshow(a[:,:,1])
    plt.show()
    plt.imshow(a[:,:,2])
    plt.show()

    print(np.min(a), np.max(a))

#print(count)

In [ ]:
a = cv2.imread("SAR_swath_images_VV+VH+WS/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,0])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()

print(np.min(a), np.max(a))
b = a/np.max(a)
print(np.min(b), np.max(b))
print("cv2_imshow")
cv2_imshow(b)
print("plt.imshow")
plt.imshow(b)
plt.show()

#c = cv2.imread("SAR_swath_images_VV+VH+VH/category1/2016-09-15 14_12_58.png")
#cv2_imshow(c)
#print(np.min(c), np.max(c))
#d = c/np.max(c)
#cv2_imshow(d)

In [ ]:
a = cv2.imread("SAR_swath_images/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,0])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()
plt.imshow(a[:,:,1])
plt.show()
plt.imshow(a[:,:,2])
plt.show()

print(np.min(a), np.max(a))

In [ ]:
a = cv2.imread("SAR_swath_images_VV+VH+WS/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,0])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()
plt.imshow(a[:,:,1])
plt.show()
plt.imshow(a[:,:,2])
plt.show()

print(np.min(a), np.max(a))

In [ ]:
a = cv2.imread("SAR_swath_images_VV+VH+VH/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,0])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()
plt.imshow(a[:,:,1])
plt.show()
plt.imshow(a[:,:,2])
plt.show()

print(np.min(a), np.max(a))

In [ ]:
a = cv2.imread("SAR_swath_images_WS+WS+WS/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,1])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()
plt.imshow(a[:,:,1])
plt.show()
plt.imshow(a[:,:,2])
plt.show()

print(np.min(a), np.max(a))

In [ ]:
a = cv2.imread("SAR_swath_images_WS+sWSO+cWSO/category1/2016-09-01 20_30_16.png")
print("cv2_imshow")
cv2_imshow(a)
print("plt.imshow")
plt.imshow(a)
plt.show()

print("feature_co")
print("cv2_imshow")
cv2_imshow(a[:,:,0])

print("plt.imshow")
plt.imshow(a[:,:,0])
plt.show()
plt.imshow(a[:,:,1])
plt.show()
plt.imshow(a[:,:,2])
plt.show()

print(np.min(a), np.max(a))

###2.2. Delete doubled files

In [ ]:
folder_to_delete = "/content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data/SAR_swath_images_VV+VH+WS_land=1/category2"
files_to_delete = glob.glob(folder_to_delete + '/*.png')

source_folder = "/content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data/SAR_swath_images_VV+VH+WS_land=1/category1"
source_files = glob.glob(source_folder + '/*.png')

for f in files_to_delete:
    f = f.split('/')[-1]
    for s in source_files:
        s = s.split('/')[-1]
        if f == s:
          print('File removed:', s)
          os.remove(source_folder + '/' + s)

In [ ]:
source_folder = "/content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data/SAR_swath_images_VV+VH+WS_land=1/category5"
source_files = glob.glob(source_folder + '/*.png')
len(source_files)

##3.Split data between train, validation and test csv files

In [ ]:
LABELS_PATH = "/content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data/labels/via_project_11Jan2021_12h22m_csv.csv"
#labels_path_file = glob.glob('{}/*.csv'.format(LABELS_PATH))
## in case there is no txt file for that image
#if labels_path_file == []:
#  print("No labels file found.")
#  sys.exit(0)

#df = pd.read_csv(labels_path_file[0])
df = pd.read_csv(LABELS_PATH)
cols = df.columns.tolist()
for idx, row in df.iterrows():
    # retrieve the coordinates of the bounding boxes
    df['region_shape_attributes'][idx] = re.findall(r'\d+', df['region_shape_attributes'][idx])

images_dir = "SAR_swath_images_VV+VH+WS_land=1/"
category = ["category1/", "category2/", "category3/", "category4/", "category5/"]

# use a seed so to obtain the same splitting every time
random.seed(20)

df_train = [] 
df_val = [] 
df_test = [] 
df_full = []

for cat in category:
    dir = images_dir + cat
    lon_dir = dir + "lon"
    lat_dir = dir + "lat"

    image_list = glob.glob('{}/*.png'.format(dir))

    for image_path in image_list:
        #print(image_path)
        image_name = os.path.basename(image_path)

        lon_list = glob.glob('{}/{}.npy'.format(lon_dir, image_name.split('.')[0]))
        lat_list = glob.glob('{}/{}.npy'.format(lat_dir, image_name.split('.')[0]))

        lon = lon_list[0] if lon_list != [] else None
        lat = lat_list[0] if lat_list != [] else None

        aux = [image_path]
        aux.extend([lat, lon])
        #print(df[df.filename == image_name])
        index = df[df.filename == image_name].index[0]
        bb_present = 1 if df['region_count'][index] == 1 else 0
        aux.append(bb_present)
        aux.append(df['region_shape_attributes'][index])

        df_full.append(aux)
        rand = random.randint(0, 100)
        # split used 60, 20, 20
        if rand < 60:
            df_train.append(aux)
        elif rand < 80:
            df_val.append(aux)
        else:
            df_test.append(aux) 

col_names = ['image', 'lat', 'lon', 'label', 'bbox_shape']
df_train = pd.DataFrame(df_train, columns = col_names)
df_val = pd.DataFrame(df_val, columns = col_names)
df_test = pd.DataFrame(df_test, columns = col_names)
df_full = pd.DataFrame(df_full, columns = col_names)

if (df_full['lat'].values == None).all() and (df_full['lon'].values == None).all():
    print("lat and lon variables not considered.")
    df_full.drop(columns=['lat', 'lon'], inplace=True)

# create path to save the csv files
CSV_PATH = images_dir + 'csv'
os.makedirs(CSV_PATH, exist_ok=True)

if df_train.empty == False:
    df_train.to_csv('{}/training.csv'.format(CSV_PATH), index = False)

if df_val.empty == False:
    df_val.to_csv('{}/val.csv'.format(CSV_PATH), index = False)

if df_test.empty == False:
    df_test.to_csv('{}/test.csv'.format(CSV_PATH), index = False)

if df_full.empty == False:
    df_full.to_csv('{}/full_dataset.csv'.format(CSV_PATH), index = False)

##4.Test the location of the bounding boxes around the eye

In [ ]:
import matplotlib
df = pd.read_csv('SAR_swath_images_VV+VH+WS_land=1/csv/full_dataset.csv', converters={'bbox_shape': eval})

for idx, row in df.iterrows():
    print("Image:", df['image'][idx], "label:", df['label'][idx])
    img = cv2.imread(df['image'][idx])

    b = img/np.max(img)
    fig, ax = plt.subplots(figsize=(5,8))
    plt.imshow(b)

    if df['bbox_shape'][idx] != []:
        bbox = df['bbox_shape'][idx]

        # read dimensions of bbox_eye
        cX = (int)(bbox[0])
        cY = (int)(bbox[1])
        bb_width = (int)(bbox[2])
        bb_height = (int)(bbox[3])

        ax.add_patch(matplotlib.patches.Rectangle((cX, cY), bb_width, bb_height, color ='green', fc='none'))
    plt.show()

##5.Link .nc files to ARCHER products

In [ ]:
main_dir = "Link_ANCHER_IFREMER"

###5.1. Read all txt files from ARCHER and convert them to csv files

In [ ]:
# directory where ARCHER products are stored
archer_dir = main_dir + "/ARCHER_products"
folders = glob.glob("{}/*".format(archer_dir))
cnt = 0

for folder in folders:
    print(folder)
    files = glob.glob("{}/*".format(folder))
    file = "{}/archerTrackTable.txt".format(folder)

    if file in files:
        cnt+=1
        try:
            # read .txt file as pandas df
            cols = ["Date/Time (UTC)", "Source Sensor", "Vmax (kts)", "t_off (hrs)", "Geo-ref Lat",
                    "Lon", "Opr Ctr Lat", "Lon2", "50% cert rad", "95% cert rad", "Eye diam (deg)", "% cert eye", "50% cert fxa"]
            df = pd.read_csv(file, sep="\s{2,}", engine='python', skiprows=1, names = cols, index_col=False)
            df["Date/Time (UTC)"] = pd.to_datetime(df["Date/Time (UTC)"], format='%Y-%m-%d %H:%M:%S')
            df.to_csv("{}/archerTrackTable.csv".format(folder), index=False)

        except ValueError as err:
            print("Error:", err)
            file = "{}/summaryTable.txt".format(folder)
            if file in files:
                try:
                    # read .txt file as pandas df
                    cols = ["Date/Time (UTC)", "Source Sensor", "Vmax (kts)", "ARCHER Lat", "Lon", "Geo-ref Lat",
                            "Lon2", "50% cert rad", "95% cert rad", "Eye diam (deg)", "% cert eye", "50% cert fxa"]
                    df = pd.read_csv(file, sep="\s{2,}", engine='python', skiprows=1, names = cols, index_col=False)
                    df["Date/Time (UTC)"] = df["Date/Time (UTC)"].apply(lambda date: date if " *" not in date else date.replace(" *", ""))
                    df["Date/Time (UTC)"] = pd.to_datetime(df["Date/Time (UTC)"], format='%Y-%m-%d %H:%M:%S')
                    df.to_csv("{}/summaryTable.csv".format(folder), index=False)
                
                except ValueError as err:
                    print("Second Error:", err)
                    continue
print(cnt)

###5.2. Link each .nc file to an ARCHER csv file

####5.2.1. Gather info from .nc files

In [ ]:
categories = ["category1", "category2", "category3", "category4", "category5"]
#category = 'category4'

nc_files = []
date_time = []
extreme_coords = []
for category in categories:
    print(category)
    # directory where .nc files are
    saved_dir = "SAR_swath_nc/{}".format(category)

    # directory where corresponding images will be saved
    images_dir = "SAR_swath_images_VV+VH+WS_dilated/{}".format(category)

    if category == 'category5':
        # get .nc files
        nc_list = glob.glob("{}/*.nc".format(saved_dir))
    else:
        cat = int(re.findall("\d", category)[0])
        categories_higher = [i for i in range(cat+1, 6)]
        nc_list_higher_all = []
        for cat_h in categories_higher:
            nc_list_higher_all.extend(glob.glob('SAR_swath_nc/category{}/*.nc'.format(cat_h)))

        nc_list_higher = [os.path.basename(x) for x in nc_list_higher_all]
        nc_list = [file for file in glob.glob("{}/*.nc".format(saved_dir)) if os.path.basename(file) not in nc_list_higher]
        
    cnt = 0
    for nc_image in nc_list:
        # read nc image
        full_info_image = netCDF4.Dataset(nc_image, mode='r') 

        # getting the information of the feature you want
        try:
            # try to read this feature as it usually gives problems
            feature_cross = full_info_image.variables["nrcs_detrend_cross"][:]

            # extract time registered
            tmax_units = datetime.datetime.strptime(full_info_image.measurementDate, '%Y-%m-%dT%H:%M:%SZ')
            #tmax_units = tmax_units.strftime('%Y-%m-%d %H_%M_%S')

            # extract longitude and latitude features
            feature_lon = full_info_image.variables["lon"][:]
            #print("feature_lon:", np.max(feature_lon[0]), np.min(feature_lon[0]))
            feature_lat = full_info_image.variables["lat"][:]
            #print("feature_lat:", np.max(feature_lat[0]), np.min(feature_lat[0]))

            coords = [np.min(feature_lat[0]), np.max(feature_lat[0]), np.min(feature_lon[0]), np.max(feature_lon[0])]

            nc_files.append(nc_image)
            date_time.append(tmax_units)
            extreme_coords.append(coords)

            full_info_image.close()
        except KeyError as err:
            # creating KeyError instance for book keeping
            print("Error:", err)
            full_info_image.close()
            continue
        cnt +=1
    print(cnt)

df = pd.DataFrame({"nc_files": nc_files, "date_time": date_time, "extreme_coords": extreme_coords})
df["image"] = df["date_time"].apply(lambda i: i.strftime('%Y-%m-%d %H_%M_%S'+'.png'))
df["ARCHER product"] = np.empty((len(df), 0)).tolist()
df.to_csv(main_dir + "/IFREMER_info.csv", index=False)
df

In [ ]:
df.dtypes

####5.2.2. Link .nc files to ARCHER

In [ ]:
df = pd.read_csv(main_dir + "/IFREMER_info.csv", converters={'ARCHER product': eval, 'extreme_coords': eval})
df["date_time"] = pd.to_datetime(df["date_time"])

# directory where ARCHER products are stored
archer_dir = main_dir + "/ARCHER_products"
folders = glob.glob("{}/*".format(archer_dir))
cnt = 0

def link_ARCHER_IFREMER(x, EndDate, StartDate, lat_arr, lon_arr):
    lat_ok = False
    lon_ok = False
    x['extreme_coords'] = np.array(x['extreme_coords'], dtype=np.float)
    print("StartDate:", StartDate, ", EndDate:", EndDate, ", SAR datetime:", x['date_time'])


    if StartDate <= x['date_time'] <= EndDate:
        print("Inside datetime!")
        print(x['extreme_coords'])

        print(lat_arr)
        if any((x['extreme_coords'][0] <= lat_arr) & (lat_arr <= x['extreme_coords'][1])):
            print("LAT OK!")
            lat_ok = True
        #for item in lat_arr:
        #    print(type(item), type(x['extreme_coords'][0]), type(x['extreme_coords'][1]))
        #    if x['extreme_coords'][0] <= item <= x['extreme_coords'][1]:
        #        print("LAT OK! Value: {}".format(item))
        #        lat_ok = True
        #        break

        print(lon_arr)
        if any((x['extreme_coords'][2] <= lon_arr) & (lon_arr <= x['extreme_coords'][3])):
            print("LON OK!")
            lon_ok = True
        #for item in lon_arr:
        #    if x['extreme_coords'][2] <= item <= x['extreme_coords'][3]:
        #        print("LON OK! Value: {}".format(item))
        #        lon_ok = True
        #        break

        if lat_ok and lon_ok:
            #sys.exit()
            return True
        else:
            return False
    else:
        return False

for folder in folders:
    cnt +=1
    print(folder)
    csv_files = glob.glob("{}/*.csv".format(folder))

    if len(csv_files) > 1:
        csv_files = [s for s in csv_files if "new" in s]

    for csv_file in csv_files:

        archer_df = pd.read_csv(csv_file, header=0)
        archer_df["Date/Time (UTC)"] = pd.to_datetime(archer_df["Date/Time (UTC)"])
        archer_df = archer_df.dropna()

        StartDate = np.min(archer_df["Date/Time (UTC)"])
        EndDate = np.max(archer_df["Date/Time (UTC)"])
        if (EndDate - StartDate ) / np.timedelta64(1, 'D') > 30:    # time difference of more than 30 days
            print(folder, "- StartDate:", StartDate, "EndDate:", EndDate)
        
        inTime_df = df[(df.date_time >= StartDate) & (df.date_time <= EndDate)]

        if not inTime_df.empty:
            lat = archer_df["Geo-ref Lat"].dropna().to_numpy()
            lat_arr = np.delete(lat, np.argwhere(lat == "***"))
            lat_arr = lat_arr.astype(np.float)

            lon_col = "Lon" if "archerTrackTable" in os.path.basename(csv_file) else "Lon2"
            lon = archer_df[lon_col].dropna().to_numpy()
            lon_arr = np.delete(lon, np.argwhere(lon == "***"))
            lon_arr = lon_arr.astype(np.float)

            #df["ARCHER product"] = df["ARCHER product"].apply(lambda i: i if not (df.date_time < EndDate & df.date_time > StartDate) else i.append(folder)
            df.apply(lambda x: x["ARCHER product"].append(csv_file) if link_ARCHER_IFREMER(x, EndDate, StartDate, lat_arr, lon_arr) else x["ARCHER product"], axis = 1)
    
df

In [ ]:
df.to_csv(main_dir + "/ARCHER_IFREMER_link_by_date_and_coords.csv", index=False)

####5.2.3. Deal with problematic ARCHER products

In [ ]:
# folders with time_diffs between StartDate and EndDate higher than 30 days (problematic)
# Link_ANCHER_IFREMER/ARCHER_products/2016_30W - StartDate: 2016-12-21 05:30:00 EndDate: 2017-11-15 11:30:00
# Link_ANCHER_IFREMER/ARCHER_products/2018_03S - StartDate: 2018-01-02 17:30:00 EndDate: 2018-11-11 17:30:00
# Link_ANCHER_IFREMER/ARCHER_products/2018_09P - StartDate: 2018-02-13 06:00:00 EndDate: 2019-01-07 21:00:00
# Link_ANCHER_IFREMER/ARCHER_products/2020_07S - StartDate: 2020-01-11 11:30:00 EndDate: 2020-12-30 17:30:00
# Link_ANCHER_IFREMER/ARCHER_products/2020_10S - StartDate: 2020-01-26 17:30:00 EndDate: 2021-01-19 11:30:00

folder = "Link_ANCHER_IFREMER/ARCHER_products/2020_10S"
csv_files = glob.glob("{}/*.csv".format(folder))

for csv_file in csv_files:

    archer_df = pd.read_csv(csv_file, header=0)
    archer_df["Date/Time (UTC)"] = pd.to_datetime(archer_df["Date/Time (UTC)"])
    archer_df = archer_df.sort_values(by=["Date/Time (UTC)"], ignore_index=True).dropna()

    # check time differences between consecutive obervations
    time_diffs = archer_df['Date/Time (UTC)'].diff().dropna()
    # get index for when time diff is above 2 days
    index = np.where(time_diffs / np.timedelta64(1, 'D') > 5)[0][0]

    df1 = archer_df.iloc[:index+1]
    df2 = archer_df.iloc[index+1:]
    print(df1)
    print(df2)

    input("Press Enter to continue...")

    df1.to_csv("{}/{}_new1.csv".format(folder, os.path.basename(csv_file)[:-4]), index=False)
    df2.to_csv("{}/{}_new2.csv".format(folder, os.path.basename(csv_file)[:-4]), index=False)

###5.3. Test location of eye according to ARCHER and best track

In [ ]:
def find_nearest(array, value):
    #array = np.asarray(array)
    X = np.abs(array - value)
    idx = np.where(X == X.min())
    print("Index:", idx)
    #print("Value wanted: {} and closest value in X: {}".format(value, array[idx[0], idx[1]]))
    #return array[idx[0], idx[1]], idx
    flat_index = np.argmin(np.abs(array - value))
    alt_idx = np.unravel_index(flat_index, array.shape)
    print("Alternatively:", alt_idx)
    print("Value wanted: {} and closest value in X: {}".format(value, array[alt_idx[0], alt_idx[1]]))
    return array[alt_idx[0], alt_idx[1]], alt_idx

In [ ]:
request_url = "https://cyclobs.ifremer.fr/app/api/getData?cat_min=cat-1&cat_max=cat-5&mission=S1B,S1A&product_type=swath&include_cols=all"
df_request = pd.read_csv(request_url)

nc_CyclObs_info = pd.DataFrame(data = {'data_url': df_request['data_url'], 'sid': df_request["sid"], 'TC_name': df_request["cyclone_name"]})
nc_CyclObs_info["data_url"] = nc_CyclObs_info["data_url"].apply(lambda x: os.path.basename(x))
nc_CyclObs_info

In [ ]:
df = pd.read_csv(main_dir + "/ARCHER_IFREMER_link_by_date_and_coords.csv", converters={'ARCHER product': eval})
df["date_time"] = pd.to_datetime(df["date_time"])

best_track_df = pd.read_csv("best_track/ibtracs.since1980.list.v04r00.csv", header=0, skiprows=range(1, 2))
best_track_df["ISO_TIME"] = pd.to_datetime(best_track_df["ISO_TIME"])
best_track_df['ISO_TIME_str'] = best_track_df['ISO_TIME'].apply(lambda i: datetime.datetime.strftime(i, "%Y-%m-%d"))

time_diffs = []

cnt = 0
for index, row in df.iterrows():
    print(row)
    cnt+=1

    full_info_image = netCDF4.Dataset(row["nc_files"], mode='r') 

    mask = full_info_image.variables["mask_flag"][:]
    mask[mask != 0] = 1
    # dilate mask
    kernel = np.ones((11, 11), np.int8)
    mask_dilated = cv2.dilate(mask[0], kernel, iterations = 1)

    feature_co = full_info_image.variables["nrcs_detrend_co"][:]
    feature_co[0][mask_dilated != 0] = 0 # mask out the land values
    feature_co = (feature_co[0] - np.min(feature_co[0]))/(np.max(feature_co[0]) - np.min(feature_co[0]))

    feature_cross = full_info_image.variables["nrcs_detrend_cross"][:]
    feature_cross[0][mask_dilated != 0] = 0 # mask out the land values
    feature_cross = (feature_cross[0] - np.min(feature_cross[0]))/(np.max(feature_cross[0]) - np.min(feature_cross[0]))

    feature_wind = full_info_image.variables["wind_speed"][:]
    feature_wind[0][mask_dilated != 0] = 0 # mask out the land values
    feature_wind = (feature_wind[0] - np.min(feature_wind[0]))/(np.max(feature_wind[0]) - np.min(feature_wind[0]))

    # stack matrices along 3rd axis (depth-wise)
    output_image = np.dstack((feature_co, feature_cross, feature_wind))

    # extract longitude and latitude features
    feature_lon = full_info_image.variables["lon"][:]
    #feature_lon = feature_lon[0]
    feature_lat = full_info_image.variables["lat"][:]
    #feature_lat = feature_lat[0]

    nc_info = nc_CyclObs_info.loc[nc_CyclObs_info["data_url"] == os.path.basename(row["nc_files"])]
    nc_sid = nc_info["sid"].values[0].upper()
    print(nc_sid)

    # prepare plot of coordinates
    fig, ax = plt.subplots()
    ax.imshow(output_image)    

    # filter best_track_df by date
    aux = best_track_df.loc[best_track_df['ISO_TIME_str'] == datetime.datetime.strftime(row["date_time"], "%Y-%m-%d")][["USA_ATCF_ID", "ISO_TIME", "LAT", "LON"]]
    #print(aux)
    if not aux.empty:
        best_track = aux.loc[aux['USA_ATCF_ID'] == nc_sid] if len(np.unique(aux['USA_ATCF_ID'])) > 1 else aux

        if not best_track.empty:
            best_track = best_track.append({"ISO_TIME": row["date_time"], "LAT": np.nan, "LON": np.nan}, ignore_index=True)
            best_track = best_track.sort_values(by=["ISO_TIME"], ignore_index=True)
            interp_best_track = best_track.interpolate(method='linear')
            bt_lat_value = interp_best_track.loc[interp_best_track["ISO_TIME"] == row["date_time"]]["LAT"]
            bt_lon_value = interp_best_track.loc[interp_best_track["ISO_TIME"] == row["date_time"]]["LON"]
            print(interp_best_track)
            print(bt_lat_value.values[0], bt_lon_value.values[0])

            _, bt_idx_lat = find_nearest(feature_lat[0], bt_lat_value.values[0])
            _, bt_idx_lon = find_nearest(feature_lon[0], bt_lon_value.values[0])
            bt_x = bt_idx_lon[1]
            bt_y = bt_idx_lat[0]
            print("BT Center:", bt_x, bt_y)

            ax.plot(bt_x, bt_y, '^', mfc='none', label = "Best track")

    for item in row['ARCHER product']:
        if row['ARCHER product'] == []:
            continue
        print(item)
        archer_df = pd.read_csv(item)
        archer_df["Date/Time (UTC)"] = pd.to_datetime(archer_df["Date/Time (UTC)"])

        lon_col = "Lon" if "archerTrackTable" in os.path.basename(item) else "Lon2"
        coords = archer_df[["Date/Time (UTC)", "Geo-ref Lat", lon_col]].dropna()
        coords_df = coords[coords["Geo-ref Lat"] != "***"]
        coords_df["Geo-ref Lat"] = coords_df["Geo-ref Lat"].astype(float)
        coords_df[lon_col] = coords_df[lon_col].astype(float)
        coords_df = coords_df.append({"Date/Time (UTC)": row["date_time"], "Geo-ref Lat": np.nan, lon_col: np.nan}, ignore_index=True)
        coords_df = coords_df.sort_values(by=["Date/Time (UTC)"], ignore_index=True)
        print(coords_df)

        # Check time difference between ARCHER records and SAR measuremente Date
        index = coords_df.index[coords_df["Date/Time (UTC)"] == row["date_time"]].tolist()[0]
        test_diff = coords_df.iloc[index-1:index+2] if index > 0 else coords_df.iloc[index:index+2]
        print(min(test_diff['Date/Time (UTC)'].diff().dropna().tolist()))
        time_diffs.append(min(test_diff['Date/Time (UTC)'].diff().dropna().tolist()))

        interp_coords_df = coords_df.interpolate(method='linear')
        lat_value = interp_coords_df.loc[interp_coords_df["Date/Time (UTC)"] == row["date_time"]]["Geo-ref Lat"]
        lon_value = interp_coords_df.loc[interp_coords_df["Date/Time (UTC)"] == row["date_time"]][lon_col]
        #print(interp_coords_df.loc[interp_coords_df["Date/Time (UTC)"] == row["date_time"]])
    
        lat, idx_lat = find_nearest(feature_lat[0], lat_value.values[0])
        lon, idx_lon = find_nearest(feature_lon[0], lon_value.values[0])
        x = idx_lon[1]
        y = idx_lat[0]
        print("ARCHER Center:", x, y)

        ax.plot(x, y, 'o', mfc='none', label = "ARCHER")
        
    ax.legend(title = "Source", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    #plots_dir = main_dir + "/plot_coords"
    #os.makedirs(plots_dir, exist_ok=True)
    #fig.savefig("{}/TC_center_by_source_{}.jpg".format(plots_dir, cnt), bbox_inches='tight')

    #if cnt == 3:
    #    break

time_diffs

In [ ]:
time_diff_df = pd.DataFrame(time_diffs, columns=['Date'])
#print(time_diff_df)

hours = time_diff_df.Date / np.timedelta64(1, 'h')
#hours = hours[hours < 48]

fig,ax = plt.subplots(1, 1, figsize=(15,8))
ax.set(xlabel = 'Time difference (hours)', title='Time difference between SAR and ARCHER products')
hours.plot.hist(bins=100)
plt.grid(True, axis='y')
plt.show()

#fig.savefig(main_dir + "/time_diff_hist.png", bbox_inches = 'tight')

##6.Test black pixels in image

In [ ]:
# read the image path
image_path = "SAR_swath_images_VV+VH+WS_dilated/category2/2018-10-24 01_06_05.png"
im = cv2.imread(image_path) # loads images as BGR in float32
image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)   # BGR -> RGB
cv2_imshow(image)
print(image.size)
print(image.shape)
print(image.shape[0]*image.shape[1])
print(cv2.countNonZero(image[:,:,0]))
print(cv2.countNonZero(image[:,:,1]))
print(cv2.countNonZero(image[:,:,2]))
percentage = (image.shape[0]*image.shape[1] - cv2.countNonZero(image[:,:,0]))/(image.shape[0]*image.shape[1]) * 100
percentage

In [ ]:
# read the image path
image_path = "SAR_swath_images_VV+VH+WS_dilated/category2/2020-09-22 10_17_34.png"
im = cv2.imread(image_path) # loads images as BGR in float32
image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)   # BGR -> RGB
cv2_imshow(image)
print(image.size)
print(image.shape)
print(image.shape[0]*image.shape[1])
print(cv2.countNonZero(image[:,:,0]))
percentage = (image.shape[0]*image.shape[1] - cv2.countNonZero(image[:,:,0]))/(image.shape[0]*image.shape[1]) * 100
percentage